In [5]:
import os

directory = "./assets/2002_4"

if not directory:
    directory = input("Input raw dataset directory:")

files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

count = 1
if files:
    curr = files[0].split("_")[0]
else:
    print("No files found in the directory.")
    exit()

for file in files:
    file_split = file.split("_")
    if file_split[0] != curr:
        curr = file_split[0]
        count += 1
    
    new_file_name = "_".join(file_split[1:])
    new_file_name = str(count).zfill(3) + "_" + new_file_name
    
    old_path = os.path.join(directory, file)
    new_path = os.path.join(directory, new_file_name)
    
    os.rename(old_path, new_path)
    print(f"Successfully changed file name from ({file}) to ({new_file_name})")

FileNotFoundError: [WinError 3] The system cannot find the path specified: './assets/2002_4'

In [6]:
import cv2
import numpy as np
import os

# Path ke folder dataset dan folder output
input_folder = "assets"
output_folder = "processed"
os.makedirs(output_folder, exist_ok=True)

def process_fingerprint(image_path, output_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply CLAHE untuk enhance kontras
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
    enhanced = clahe.apply(image)
    
    # Gaussian Blur untuk meredam noise
    blurred = cv2.GaussianBlur(enhanced, (3,3), 0)
    
    # Adaptive Thresholding untuk segmentasi
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 19, 7)
    
    # Morphological Operations untuk membersihkan noise
    kernel = np.ones((2,2), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # Fungsi skeletonisasi
    def skeletonize(img):
        skel = np.zeros(img.shape, np.uint8)
        element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
        while True:
            open_img = cv2.morphologyEx(img, cv2.MORPH_OPEN, element)
            temp = cv2.subtract(img, open_img)
            eroded = cv2.erode(img, element)
            skel = cv2.bitwise_or(skel, temp)
            img = eroded.copy()
            if cv2.countNonZero(img) == 0:
                break
        return skel
    
    skeleton_fingerprint = skeletonize(cleaned)
    inverted = cv2.bitwise_not(skeleton_fingerprint)
    
    # Simpan hasil akhir
    cv2.imwrite(output_path, inverted)

# Loop untuk proses semua gambar dalam folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        process_fingerprint(input_path, output_path)
        print(f"Processed: {filename}")

Processed: 001_1.tif
Processed: 001_2.tif
Processed: 001_3.tif
Processed: 001_4.tif
Processed: 001_5.tif
Processed: 001_6.tif
Processed: 001_7.tif
Processed: 001_8.tif
Processed: 002_1.tif
Processed: 002_2.tif
Processed: 002_3.tif
Processed: 002_4.tif
Processed: 002_5.tif
Processed: 002_6.tif
Processed: 002_7.tif
Processed: 002_8.tif
Processed: 003_1.tif
Processed: 003_2.tif
Processed: 003_3.tif
Processed: 003_4.tif
Processed: 003_5.tif
Processed: 003_6.tif
Processed: 003_7.tif
Processed: 003_8.tif
Processed: 004_1.tif
Processed: 004_2.tif
Processed: 004_3.tif
Processed: 004_4.tif
Processed: 004_5.tif
Processed: 004_6.tif
Processed: 004_7.tif
Processed: 004_8.tif
Processed: 005_1.tif
Processed: 005_2.tif
Processed: 005_3.tif
Processed: 005_4.tif
Processed: 005_5.tif
Processed: 005_6.tif
Processed: 005_7.tif
Processed: 005_8.tif
Processed: 006_1.tif
Processed: 006_2.tif
Processed: 006_3.tif
Processed: 006_4.tif
Processed: 006_5.tif
Processed: 006_6.tif
Processed: 006_7.tif
Processed: 00

In [3]:
pip install opencv-python

     ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
     ---------------------------------------- 0.2/39.5 MB 4.6 MB/s eta 0:00:09
     - -------------------------------------- 1.0/39.5 MB 13.0 MB/s eta 0:00:03
     --- ------------------------------------ 3.9/39.5 MB 31.3 MB/s eta 0:00:02
     ------ --------------------------------- 6.0/39.5 MB 35.1 MB/s eta 0:00:01
     ------- -------------------------------- 7.6/39.5 MB 37.2 MB/s eta 0:00:01
     --------- ------------------------------ 9.2/39.5 MB 34.6 MB/s eta 0:00:01
     ---------- ---------------------------- 10.3/39.5 MB 34.4 MB/s eta 0:00:01
     ------------ -------------------------- 12.5/39.5 MB 40.9 MB/s eta 0:00:01
     --------------- ----------------------- 15.9/39.5 MB 43.7 MB/s eta 0:00:01
     ------------------ -------------------- 18.4/39.5 MB 46.7 MB/s eta 0:00:01
     ------------------ -------------------- 18.9/39.5 MB 40.9 MB/s eta 0:00:01
     ------------------- ------------------- 19.4

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import shutil
import random

# Define paths
dataset_folder = "processed"
train_folder = "dataset/train"
test_folder = "dataset/test"

# Create train and test folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(dataset_folder) if f.endswith(".tif")]

# Shuffle to ensure randomness
random.shuffle(image_files)

# Split into 80% training and 20% testing
split_index = int(0.8 * len(image_files))
train_files = image_files[:split_index]
test_files = image_files[split_index:]

# Function to move files
def move_files(file_list, destination_folder):
    for file in file_list:
        src = os.path.join(dataset_folder, file)
        dest = os.path.join(destination_folder, file)
        shutil.move(src, dest)

# Move files to respective folders
move_files(train_files, train_folder)
move_files(test_files, test_folder)

print(f"Dataset split completed: {len(train_files)} train, {len(test_files)} test")

Dataset split completed: 512 train, 128 test


In [8]:
import cv2
import numpy as np
import os
import random

# Paths
base_dir = "dataset"
test_dir = os.path.join(base_dir, "test")
noisy_test_dir = os.path.join(base_dir, "noisy_test")

# Creating folders for different noise types
salt_pepper_dir = os.path.join(noisy_test_dir, "salt_pepper")
block_damage_dir = os.path.join(noisy_test_dir, "block_damage")
combined_dir = os.path.join(noisy_test_dir, "combined")

os.makedirs(salt_pepper_dir, exist_ok=True)
os.makedirs(block_damage_dir, exist_ok=True)
os.makedirs(combined_dir, exist_ok=True)

# Function to add Salt & Pepper noise
def add_salt_pepper_noise(image, salt_prob=0.02, pepper_prob=0.02):
    noisy = np.copy(image)
    num_salt = int(salt_prob * image.size)
    num_pepper = int(pepper_prob * image.size)

    # Salt (white pixels)
    coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape]
    noisy[coords[0], coords[1]] = 255

    # Pepper (black pixels)
    coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape]
    noisy[coords[0], coords[1]] = 0

    return noisy

# Function to apply Block Damage
def apply_block_damage(image, block_size=40, num_blocks=10):
    damaged = np.copy(image)
    height, width = damaged.shape[:2]

    for _ in range(num_blocks):
        x = random.randint(0, width - block_size)
        y = random.randint(0, height - block_size)
        damaged[y:y+block_size, x:x+block_size] = 0  # Black blocks

    return damaged

# Get all test images
test_images = [f for f in os.listdir(test_dir) if f.endswith(".tif")]
test_images.sort()  # Ensure consistency in selection
split_index = len(test_images) // 2  # Split 50/50

# Process images
for i, filename in enumerate(test_images):
    image_path = os.path.join(test_dir, filename)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if i < split_index:  # First 50% → Salt & Pepper
        noisy_image = add_salt_pepper_noise(image)
        cv2.imwrite(os.path.join(salt_pepper_dir, filename), noisy_image)
    else:  # Second 50% → Block Damage
        noisy_image = apply_block_damage(image)
        cv2.imwrite(os.path.join(block_damage_dir, filename), noisy_image)

    # Combined noise
    combined_image = add_salt_pepper_noise(noisy_image)
    cv2.imwrite(os.path.join(combined_dir, filename), combined_image)

    print(f"Processed: {filename}")

print("Noisy test dataset generated successfully!")

Processed: 001_8.tif
Processed: 002_1.tif
Processed: 002_4.tif
Processed: 002_6.tif
Processed: 003_1.tif
Processed: 003_4.tif
Processed: 004_8.tif
Processed: 005_1.tif
Processed: 005_7.tif
Processed: 005_8.tif
Processed: 006_2.tif
Processed: 006_3.tif
Processed: 006_5.tif
Processed: 008_1.tif
Processed: 009_1.tif
Processed: 010_6.tif
Processed: 011_1.tif
Processed: 012_3.tif
Processed: 012_7.tif
Processed: 013_2.tif
Processed: 013_5.tif
Processed: 014_1.tif
Processed: 014_2.tif
Processed: 014_4.tif
Processed: 014_6.tif
Processed: 016_4.tif
Processed: 016_5.tif
Processed: 017_4.tif
Processed: 017_8.tif
Processed: 018_3.tif
Processed: 018_7.tif
Processed: 019_1.tif
Processed: 019_3.tif
Processed: 019_6.tif
Processed: 020_3.tif
Processed: 020_6.tif
Processed: 020_8.tif
Processed: 022_5.tif
Processed: 023_4.tif
Processed: 023_7.tif
Processed: 025_3.tif
Processed: 025_5.tif
Processed: 025_8.tif
Processed: 026_3.tif
Processed: 026_4.tif
Processed: 027_1.tif
Processed: 027_5.tif
Processed: 02